In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.metrics import roc_auc_score, accuracy_score

# 1. Imports từ H2O
import h2o
from h2o.automl import H2OAutoML

# --- 1. Khởi tạo H2O Cluster ---
# Bước này bắt buộc: H2O sẽ khởi chạy một cluster Java local
# để xử lý tất cả công việc.
h2o.init(max_mem_size="4G") # Giới hạn H2O dùng 4GB RAM

# --- 2. Tải và chuẩn bị dữ liệu ---

# Tải bộ dữ liệu Titanic
data = sns.load_dataset('titanic')

# Chỉ lấy các cột cơ bản. 
# H2O SẼ TỰ ĐỘNG XỬ LÝ NaNs TRONG 'age' VÀ 'embarked'
data_subset = data[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

# Chuyển đổi pandas.DataFrame sang H2OFrame
# H2O có định dạng dữ liệu riêng để tối ưu tốc độ
print("Chuyển đổi DataFrame sang H2OFrame...")
hf = h2o.H2OFrame(data_subset)

# **BƯỚC QUAN TRỌNG:** # Chỉ định cột target là 'factor' (categorical) cho bài toán classification
# Nếu không làm việc này, H2O sẽ nghĩ đây là bài toán hồi quy (regression)
hf['survived'] = hf['survived'].asfactor()

# Định nghĩa tên cột target và các cột features
y = 'survived'
x = hf.columns
x.remove(y)

# Chia H2OFrame thành 3 tập (Train, Valid, Test)
# H2O AutoML sẽ tự dùng Train/Valid để huấn luyện và đánh giá.
# Chúng ta giữ lại tập Test để kiểm tra cuối cùng.
train, test = hf.split_frame(ratios=[0.8], seed=42)

print(f"Kích thước tập Train: {train.shape}")
print(f"Kích thước tập Test: {test.shape}")


# --- 3. Thiết lập và Huấn luyện (Train) H2O AutoML ---

print("\nBắt đầu huấn luyện H2O AutoML (Quá trình này sẽ mất vài phút)...")
# Khởi tạo H2OAutoML
# max_runtime_secs: Tổng thời gian (giây) cho phép AutoML chạy.
aml = H2OAutoML(max_runtime_secs=300, # Chạy trong 5 phút
                project_name="titanic_automl",
                seed=42)

# Huấn luyện mô hình
# H2O sẽ tự động dùng 'train' và 'validation' (nếu có)
aml.train(x=x, y=y, training_frame=train)


# --- 4. Đánh giá (Evaluate) và xem kết quả ---

# Lấy ra bảng xếp hạng (Leaderboard) của tất cả các mô hình
lb = aml.leaderboard
print("\n--- Bảng xếp hạng AutoML ---")
# 'head()' sẽ hiển thị các mô hình tốt nhất
# H2O thường sẽ có các mô hình 'StackedEnsemble' (kết hợp nhiều mô hình)
print(lb.head(rows=lb.nrows))

# Lấy ra mô hình tốt nhất (gọi là 'leader')
leader_model = aml.leader

# Đánh giá mô hình tốt nhất trên tập Test
print("\nĐánh giá mô hình tốt nhất trên tập Test...")
performance = leader_model.model_performance(test)
# print(performance) # In ra rất nhiều chỉ số

# Dự đoán trên tập Test
preds = leader_model.predict(test)

# 'preds' là một H2OFrame. Cần chuyển về pandas để dùng sklearn
preds_df = preds.as_data_frame()
test_df = test.as_data_frame()

# Lấy ra nhãn thật
y_true = test_df['survived']

# Lấy ra xác suất dự đoán cho class 1 (Survived)
test_probs_class_1 = preds_df['p1']

# Lấy ra nhãn dự đoán (0 hoặc 1)
y_pred = preds_df['predict']

# In kết quả
auc_score = roc_auc_score(y_true, test_probs_class_1)
acc_score = accuracy_score(y_true, y_pred)

print("--- KẾT QUẢ ĐÁNH GIÁ (H2O) ---")
print(f"ROC-AUC trên tập Test: {auc_score:.4f}")
print(f"Accuracy trên tập Test: {acc_score:.4f}")